In [46]:
import pandas as pd 
import numpy as np 
import re
import nltk
from nltk.corpus import stopwords

In [47]:
df = pd.read_csv("./clean_data1.csv")
print(df.shape)
#df.head()
df = df[["ID","Category","newer_res","Resume"]]
df['newer_res'].replace('', np.nan, inplace=True)
df.dropna(subset=['newer_res'], inplace=True)
df.shape

(1212, 5)


(1199, 4)

In [48]:
df.head()

,ID,Category,newer_res,Resume
0,1,HR,john h. smith p.h.r 800-991-5187 po box 1673 c...,"b'John H. Smith, P.H.R.\n800-991-5187 | PO Box..."
1,2,HR,name surname address mobile no/email personal ...,b'Name Surname\nAddress\nMobile No/Email\nPERS...
2,3,HR,anthony brown hr assistant areas of expertise ...,b'Anthony Brown\nHR Assistant\nAREAS OF EXPERT...
3,4,HR,www.downloadmela.com satheesh email id career ...,b'www.downloadmela.com\nSatheesh\nEMAIL ID:\nC...
4,5,HR,human resources director expert in organizatio...,"b""HUMAN RESOURCES DIRECTOR\n\xef\x82\xb7Expert..."


In [7]:
def extract_dob(s):
	dobre = re.compile(r'(\d{2})[/.-](\d{2})[/.-](\d{4})$')
	dob = dobre.findall(x)
	return dob

def extract_email(s):
	emailre = re.compile("([\w\-\.]+@(?:\w[\w\-]+\.)+[\w\-]+)")
	email = emailre.findall(x)
	return email

def extract_phno(s):
	phnore = re.compile("(\d{3}[-\.\s]??\d{3}[-\.\s]??\d{4}|\(\d{3}\)\s*\d{3}[-\.\s]??\d{4}|\d{3}[-\.\s]??\d{4})")#("[0-9][0-9][0-9].-?.[0-9][0-9][0-9][0-9][0-9][0-9]")
	phno = phnore.findall(x)
	return phno

def rm_stopwords(s):
    sw = set(stopwords.words("english"))
    #token_words
    text_no_sw = [i for i in s if i not in sw]
    return text_no_sw

In [49]:
text = df["newer_res"][1]

In [50]:
df["Category"].unique().shape

(25,)

In [51]:
token = nltk.word_tokenize(text)
use_data = nltk.pos_tag(token)


In [52]:
from collections import Counter
window = 2

class_dict = {}

p=0
for ind,i in enumerate(df.itertuples()):
    #if(ind>2): break
    count = {}
    ngram = {}
    flag = 0
    if i[2] not in class_dict:
        class_dict[i[2]] = [] #remove try except. 
    
    try:
        #print("Hi")
        token = nltk.word_tokenize(i[3])
        token = rm_stopwords(token)
        #print(token)
        for j in range(window,len(token)):
            #print("Ji")
            val = tuple(token[j-2:j])
            if token[j] not in count:
                count[token[j]] = 1
            else: count[token[j]]+=1
            
            if val not in ngram:
                ngram[val] = 1
            else:
                ngram[val]+=1
            #print(ngram)
                #ngrams_prob = [j for (i,j) in ngram.items()]
        class_dict[i[2]].append(ngram)
        #print(ngram)
        #print(ngram)
    except:
        pass

In [53]:
def tf(doc):
    length = len(doc)
    return {i:j/float(length) for (i,j) in doc.items() }

def idf(doc_list):
    idf_dict = {}
    N = len(doc_list)
    for i in doc_list:
        for (k,v) in i.items():
            if k in idf_dict:
                idf_dict[k]+=1
            else:
                idf_dixt[k] = 1
    for (k,v) in idf_dict:
        idf_dict[k] = math.log10(N/float(idf_dict[k]))
    return 

In [55]:
tfidf = TfidfVectorizer(sublinear_tf=True, ngram_range=(1,2), stop_words='english')
features = tfidf.fit_transform(df.newer_res)

In [40]:
from nltk import trigrams
token = nltk.word_tokenize(df["newer_res"][3])
bigram  = trigrams(token)

In [57]:
sorted(list(ngram.items()),key=lambda x:x[1],reverse=True)

[(('chicago', 'il'), 39),
 (('depaul', 'university'), 18),
 (('transferable', 'skills'), 10),
 (('university', 'chicago'), 10),
 (('month', 'year'), 9),
 (('career', 'center'), 8),
 (('chronological', 'resume'), 8),
 (('public', 'relations'), 8),
 (('peer', 'career'), 7),
 (('job', 'title'), 7),
 (('community', 'service'), 7),
 (('2008', 'present'), 7),
 (('2009', 'present'), 7),
 (('action', 'verbs'), 6),
 (('bullet', 'points'), 6),
 (('city', 'state'), 6),
 (('career', 'advisor'), 5),
 (('work', 'history'), 5),
 (('study', 'abroad'), 5),
 (('education', 'depaul'), 5),
 (('resume', 'samples'), 4),
 (('make', 'sure'), 4),
 (('organization', 'job'), 4),
 (('state', 'month'), 4),
 (('year', 'month'), 4),
 (('bulleted', 'accomplishment'), 4),
 (('additional', 'experience'), 4),
 (('bachelor', 'arts'), 4),
 (('golden', 'key'), 4),
 (('il', '60614'), 4),
 (('abroad', 'program'), 4),
 (('2006', 'present'), 4),
 (('depaul', 'community'), 4),
 (('il', 'august'), 4),
 (('career', 'advisors'), 3

In [58]:
list(ngram.items())

[(('entry', 'level'), 1),
 (('level', 'resume'), 1),
 (('resume', 'guide'), 1),
 (('guide', 'packet'), 1),
 (('packet', 'intended'), 1),
 (('intended', 'serve'), 1),
 (('serve', 'starting'), 1),
 (('starting', 'point'), 1),
 (('point', 'creating'), 1),
 (('creating', 'improving'), 1),
 (('improving', 'resume'), 2),
 (('resume', 'includes'), 1),
 (('includes', 'sample'), 1),
 (('sample', 'resumes'), 1),
 (('resumes', 'well'), 1),
 (('well', 'best'), 1),
 (('best', 'practices'), 2),
 (('practices', 'helpful'), 1),
 (('helpful', 'resources'), 1),
 (('resources', 'developing'), 1),
 (('developing', 'successful'), 1),
 (('successful', 'document'), 1),
 (('document', 'depaul'), 1),
 (('depaul', 'university'), 18),
 (('university', 'peer'), 1),
 (('peer', 'career'), 7),
 (('career', 'advisors'), 3),
 (('advisors', 'available'), 1),
 (('available', 'walk-in'), 2),
 (('walk-in', 'basis'), 2),
 (('basis', 'assist'), 1),
 (('assist', 'creating'), 1),
 (('creating', 'and/or'), 1),
 (('and/or', 'im